# 1. 데이터 전처리


In [1]:
import numpy as np
import pandas as pd
from string import punctuation

from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
df=pd.read_csv('data/ArticlesMay2017.csv', encoding='latin')
df

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,5906b3197c459f24986dd003,2412,By IAN JOHNSON,article,My Beijing: The Sacred City,"['Travel and Vacations', 'Beijing (China)', 'R...",3,Travel,1,2017-05-01 04:01:21,Unknown,"This metropolis was once a total work of art, ...",The New York Times,News,https://www.nytimes.com/2017/05/01/travel/beij...
1,NaN,5906b3297c459f24986dd004,2318,By EMMA G. FITZSIMMONS,article,"6 Million Riders a Day, 1930s Technology","['Subways', 'Delays (Transportation)', 'Transi...",68,Metro,1,2017-05-01 04:01:33,Unknown,New Yorkâs subway is struggling with old inf...,The New York Times,News,https://www.nytimes.com/2017/05/01/nyregion/ne...
2,NaN,5906ceec7c459f24986dd021,1795,By MARC TRACY,article,Seeking a Cross-Border Conference,"['Cetys University', 'College Athletics', 'Nat...",3,Sports,1,2017-05-01 06:00:05,Unknown,Cetys University is making an ambitious bid to...,The New York Times,News,https://www.nytimes.com/2017/05/01/sports/mexi...
3,NaN,5906cfa77c459f24986dd022,213,By SHANNON DOYNE,article,"Questions for: âDespite the âYuck Factor,â...",[],3,Learning,0,2017-05-01 06:03:03,Unknown,How are leeches used to treat various medical ...,The New York Times,News,https://www.nytimes.com/2017/05/01/learning/qu...
4,NaN,5906e1c07c459f24986dd039,1342,By JASON STANLEY,article,Who Is a âCriminalâ?,"['Illegal Immigration', 'Traffic and Parking V...",3,OpEd,0,2017-05-01 07:20:26,Unknown,Justice Roberts was right. The Trump administr...,The New York Times,Op-Ed,https://www.nytimes.com/2017/05/01/opinion/who...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,NaN,593087d17c459f24986e2ed8,609,By DEB AMLEN,article,"Yes, You Can Write More Than One Letter in a S...",['Crossword Puzzles'],65,Games,0,2017-06-01 21:31:55,Unknown,Rebuses are crossword elements where solvers a...,The New York Times,News,https://www.nytimes.com/2017/06/01/crosswords/...
992,NaN,59308fdc7c459f24986e2ef2,797,By PAUL KRUGMAN,article,Trumpâs Act Of Gratuitous Destruction,"['United States Economy', 'United Nations Fram...",3,OpEd,25,2017-06-01 22:06:15,Unknown,"To the president and the right, itâs all abo...",The New York Times,Op-Ed,https://www.nytimes.com/2017/06/01/opinion/tru...
993,NaN,5930a48f7c459f24986e2f24,945,By GINIA BELLAFANTE,article,"Petty Crime, Daunting Bail","['Bail', 'Robberies and Thefts', 'Poverty', 'L...",3,Metropolitan,1,2017-06-01 23:34:14,Unknown,Using conflict resolution and social services ...,The New York Times,News,https://www.nytimes.com/2017/06/01/nyregion/ge...
994,NaN,5930aa877c459f24986e2f2a,141,By BARBARA BOLTON,article,Unknown,"['Bay Ridge (Brooklyn, NY)']",0,Metro,22,2017-06-02 00:00:04,Unknown,A bus rider wants a tale to conclude before he...,The New York Times,News,https://www.nytimes.com/2017/06/01/nyregion/me...


In [3]:
print('열의 개수: ', len(df.columns))
print(df.columns)

열의 개수:  16
Index(['abstract', 'articleID', 'articleWordCount', 'byline', 'documentType',
       'headline', 'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [4]:
df['headline'].isnull().values.any()

False

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   abstract          33 non-null     object
 1   articleID         996 non-null    object
 2   articleWordCount  996 non-null    int64 
 3   byline            996 non-null    object
 4   documentType      996 non-null    object
 5   headline          996 non-null    object
 6   keywords          996 non-null    object
 7   multimedia        996 non-null    int64 
 8   newDesk           996 non-null    object
 9   printPage         996 non-null    int64 
 10  pubDate           996 non-null    object
 11  sectionName       996 non-null    object
 12  snippet           996 non-null    object
 13  source            996 non-null    object
 14  typeOfMaterial    996 non-null    object
 15  webURL            996 non-null    object
dtypes: int64(3), object(13)
memory usage: 124.6+ KB


In [6]:
headline = [title for title in df.headline.values]
headline[:5]

['My Beijing: The Sacred City',
 '6 Million Riders a Day, 1930s Technology',
 'Seeking a Cross-Border Conference',
 'Questions for: â\x80\x98Despite the â\x80\x9cYuck Factor,â\x80\x9d Leeches Are Big in Russian Medicineâ\x80\x99',
 'Who Is a â\x80\x98Criminalâ\x80\x99?']

In [7]:
len(headline)

996

In [8]:
# 노이즈 데이터('Unknown') 제거
headline = [title for title in headline if title != 'Unknown']
len(headline)

935

In [9]:
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def repreprocessing(s):
    s=s.encode("utf8").decode("ascii", 'ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [11]:
text = [repreprocessing(x) for x in headline]
text[:5]

['my beijing the sacred city',
 '6 million riders a day 1930s technology',
 'seeking a crossborder conference',
 'questions for despite the yuck factor leeches are big in russian medicine',
 'who is a criminal']

In [12]:
t = Tokenizer()
t.fit_on_texts(text)
vocab_size = len(t.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 2653


In [13]:
sequences = []
for line in text: 
    encoded = t.texts_to_sequences([line])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[48, 799],
 [48, 799, 1],
 [48, 799, 1, 800],
 [48, 799, 1, 800, 57],
 [114, 406],
 [114, 406, 407],
 [114, 406, 407, 2],
 [114, 406, 407, 2, 179],
 [114, 406, 407, 2, 179, 801],
 [114, 406, 407, 2, 179, 801, 802],
 [803, 2]]

In [14]:
print(t.word_index)

{'the': 1, 'a': 2, 'to': 3, 'of': 4, 'in': 5, 'for': 6, 'and': 7, 'trump': 8, 'is': 9, 'on': 10, 'new': 11, 'how': 12, 'with': 13, 'you': 14, 'at': 15, 'an': 16, 'season': 17, 'what': 18, 'it': 19, 'its': 20, 'york': 21, 'episode': 22, 'trumps': 23, 'from': 24, 'as': 25, 'be': 26, 'not': 27, 'are': 28, 'may': 29, 'when': 30, 'but': 31, 'after': 32, 'about': 33, '3': 34, 'today': 35, 'this': 36, 'us': 37, 'recap': 38, 'by': 39, 'we': 40, 'comey': 41, 'house': 42, 'questions': 43, 'health': 44, 'president': 45, 'bill': 46, 'that': 47, 'my': 48, 'too': 49, 'get': 50, 'was': 51, 'life': 52, 'or': 53, 'will': 54, 'why': 55, 'your': 56, 'city': 57, 'dont': 58, 'care': 59, 'more': 60, 'do': 61, 'can': 62, 'has': 63, 'no': 64, 'up': 65, 'time': 66, 'out': 67, 'into': 68, 'better': 69, 'war': 70, 'his': 71, 'their': 72, 'says': 73, 'american': 74, 'love': 75, 'one': 76, 'gop': 77, '5': 78, 'climate': 79, 'should': 80, 'donald': 81, 'getting': 82, 'mr': 83, 'james': 84, 'like': 85, 'first': 86, 

In [15]:
index_to_word = {}
for key, value in t.word_index.items():
    index_to_word[value] = key

print('빈도수 상위 1번 단어 :', index_to_word[1])
print('빈도수 상위 582번 단어 :', index_to_word[582])

빈도수 상위 1번 단어 : the
빈도수 상위 582번 단어 : taiwan


In [16]:
max_len=max(len(s) for s in sequences)
print('샘플의 최대 길이 :', max_len)

샘플의 최대 길이 : 16


In [17]:
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0  48 799]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0  48 799   1]
 [  0   0   0   0   0   0   0   0   0   0   0   0  48 799   1 800]]


In [18]:
X = sequences[:,:-1]
y = sequences[:,-1]

In [19]:
y = to_categorical(y, num_classes=vocab_size)

In [20]:
X.shape, y.shape

((5501, 15), (5501, 2653))

In [21]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM

In [22]:
embedding = Embedding(vocab_size, 10, input_length=max_len-1, name="Embedding_Layer")
lstm = LSTM(128, name="LSTM_Layer")
output = Dense(vocab_size, activation='softmax', name="Output_Layer")

In [23]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1, 
                    name="Embedding_Layer"))
model.add(LSTM(128, name="LSTM_Layer"))
model.add(Dense(vocab_size, activation='softmax', name="Output_Layer"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Embedding_Layer (Embedding)  (None, 15, 10)            26530     
_________________________________________________________________
LSTM_Layer (LSTM)            (None, 128)               71168     
_________________________________________________________________
Output_Layer (Dense)         (None, 2653)              342237    
Total params: 439,935
Trainable params: 439,935
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [25]:
history = model.fit(X, y, epochs=200, verbose=2)

C:\ProgramData\Anaconda3\envs\tf2\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
 - 2s - loss: 7.4876 - accuracy: 0.0294
Epoch 2/200
 - 2s - loss: 6.9882 - accuracy: 0.0369
Epoch 3/200
 - 2s - loss: 6.8713 - accuracy: 0.0358
Epoch 4/200
 - 2s - loss: 6.7784 - accuracy: 0.0394
Epoch 5/200
 - 2s - loss: 6.6673 - accuracy: 0.0453
Epoch 6/200
 - 2s - loss: 6.5267 - accuracy: 0.0500
Epoch 7/200
 - 2s - loss: 6.3686 - accuracy: 0.0524
Epoch 8/200
 - 2s - loss: 6.2013 - accuracy: 0.0556
Epoch 9/200
 - 2s - loss: 6.0303 - accuracy: 0.0564
Epoch 10/200
 - 2s - loss: 5.8651 - accuracy: 0.0656
Epoch 11/200
 - 2s - loss: 5.7068 - accuracy: 0.0687
Epoch 12/200
 - 2s - loss: 5.5599 - accuracy: 0.0749
Epoch 13/200
 - 2s - loss: 5.4194 - accuracy: 0.0823
Epoch 14/200
 - 2s - loss: 5.2850 - accuracy: 0.0836
Epoch 15/200
 - 2s - loss: 5.1537 - accuracy: 0.0882
Epoch 16/200
 - 2s - loss: 5.0333 - accuracy: 0.0953
Epoch 17/200
 - 2s - loss: 4.9112 - accuracy: 0.0971
Epoch 18/200
 - 2s - loss: 4.7944 - accuracy: 0.1074
Epoch 19/200
 - 2s - loss: 4.6782 - accuracy: 0.1196
Ep

Epoch 155/200
 - 2s - loss: 0.4101 - accuracy: 0.8927
Epoch 156/200
 - 2s - loss: 0.4069 - accuracy: 0.8933
Epoch 157/200
 - 2s - loss: 0.4033 - accuracy: 0.8917
Epoch 158/200
 - 2s - loss: 0.3995 - accuracy: 0.8929
Epoch 159/200
 - 2s - loss: 0.3968 - accuracy: 0.8920
Epoch 160/200
 - 2s - loss: 0.4119 - accuracy: 0.8926
Epoch 161/200
 - 2s - loss: 0.4007 - accuracy: 0.8924
Epoch 162/200
 - 2s - loss: 0.3879 - accuracy: 0.8931
Epoch 163/200
 - 2s - loss: 0.3852 - accuracy: 0.8955
Epoch 164/200
 - 2s - loss: 0.3825 - accuracy: 0.8942
Epoch 165/200
 - 2s - loss: 0.3796 - accuracy: 0.8951
Epoch 166/200
 - 2s - loss: 0.3765 - accuracy: 0.8942
Epoch 167/200
 - 2s - loss: 0.3728 - accuracy: 0.8951
Epoch 168/200
 - 2s - loss: 0.3716 - accuracy: 0.8960
Epoch 169/200
 - 2s - loss: 0.3713 - accuracy: 0.8951
Epoch 170/200
 - 2s - loss: 0.3680 - accuracy: 0.8946
Epoch 171/200
 - 2s - loss: 0.3663 - accuracy: 0.8955
Epoch 172/200
 - 2s - loss: 0.3637 - accuracy: 0.8957
Epoch 173/200
 - 2s - loss: 

In [26]:
def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence = ''
    for _ in range(n): 
        encoded = t.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=15, padding='pre')
        result = model.predict_classes(encoded, verbose=0)
        for word, index in t.word_index.items(): 
            if index == result:
                break
        current_word = current_word + ' '  + word
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

In [40]:
print(sentence_generation(model, t, 'i', 9))
df['headline'][513]

i accidentally killed a child may i contact the family


'I Accidentally Killed a Child. May I Contact The Family?'

In [46]:
print(sentence_generation(model, t, 'how', 7))
df['headline'][311]

how to be mindful while taking a shower


'How to Be Mindful While Taking a Shower'

In [38]:
print(sentence_generation(model, t, 'what', 5))
df['headline'][948]

what you get for 1900000 fbi


'What You Get for â\x80¦ $1,900,000'